<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# SAR Single Node on MovieLens (Python, CPU)

Simple Algorithm for Recommendation (SAR) is a fast and scalable algorithm for personalized recommendations based on user transaction history. It produces easily explainable and interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. SAR is a kind of neighborhood based algorithm (as discussed in [Recommender Systems by Aggarwal](https://dl.acm.org/citation.cfm?id=2931100)) which is intended for ranking top items for each user. More details about SAR can be found in the [deep dive notebook](../02_model_collaborative_filtering/sar_deep_dive.ipynb). 

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users that interacted with one item are also likely to have interacted with the other. A user has an ***affinity*** to an item if they have interacted with it in the past.

### Advantages of SAR:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time. 
- Fast scoring, only involving multiplication of the similarity matrix with an affinity vector

### Notes to use SAR properly:
- Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
- It's memory-hungry, requiring the creation of an $mxm$ sparse square matrix (where $m$ is the number of items). This can also be a problem for many matrix factorization algorithms.
- SAR favors an implicit rating scenario and it does not predict ratings.

This notebook provides an example of how to utilize and evaluate SAR in Python on a CPU.

# 0 Global Settings and Imports

In [18]:
!pip3 install recommenders

Defaulting to user installation because normal site-packages is not writeable


In [19]:
import sys
import logging
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.python_utils import binarize
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.sar import SAR
from recommenders.evaluation.python_evaluation import (
    map,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)
from recommenders.utils.notebook_utils import store_metadata

%load_ext autoreload
%autoreload 2

print(f"System version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
System version: 3.9.6 (default, Apr 30 2025, 02:07:17) 
[Clang 17.0.0 (clang-1700.0.13.5)]
NumPy version: 2.0.2
Pandas version: 2.3.3


# 1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>,<Time>,[<Event Type>], [<Event Weight>]`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. Each event type can be assigned a different weight, for example, we might assign a “buy” event a weight of 10, while a “view” event mig－ht only have a weight of 1.

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [ ]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = "100k"

### 1.1 Download and use the MovieLens Dataset

In [21]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE
)

# Convert the float precision to 32-bit in order to reduce memory consumption 
data["rating"] = data["rating"].astype(np.float32)

data.head()

2025-12-24 16:02:47,922 DEBUG    Starting new HTTPS connection (1): files.grouplens.org:443
2025-12-24 16:02:48,714 DEBUG    https://files.grouplens.org:443 "GET /datasets/movielens/ml-100k.zip HTTP/1.1" 200 4924029
2025-12-24 16:02:48,716 INFO     Downloading https://files.grouplens.org/datasets/movielens/ml-100k.zip
100%|██████████| 4.81k/4.81k [00:02<00:00, 2.09kKB/s]


,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


### 1.2 Split the data using the python random splitter provided in utilities:

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because SAR generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.

In [22]:
train, test = python_stratified_split(data, ratio=0.75, col_user="userID", col_item="itemID", seed=42)


In [23]:
print("""
Train:
Total Ratings: {train_total}
Unique Users: {train_users}
Unique Items: {train_items}

Test:
Total Ratings: {test_total}
Unique Users: {test_users}
Unique Items: {test_items}
""".format(
    train_total=len(train),
    train_users=len(train['userID'].unique()),
    train_items=len(train['itemID'].unique()),
    test_total=len(test),
    test_users=len(test['userID'].unique()),
    test_items=len(test['itemID'].unique()),
))


Train:
Total Ratings: 74992
Unique Users: 943
Unique Items: 1653

Test:
Total Ratings: 25008
Unique Users: 943
Unique Items: 1444



# 2 Train the SAR Model

### 2.1 Instantiate the SAR algorithm and set the index

We will use the single node implementation of SAR and specify the column names to match our dataset (timestamp is an optional column that is used and can be removed if your dataset does not contain it).

Other options are specified to control the behavior of the algorithm as described in the [deep dive notebook](../02_model_collaborative_filtering/sar_deep_dive.ipynb).

In [24]:
# Configure logging to show debug-level messages with timestamps and severity. 
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

#set up default col for SAR
model = SAR(
    col_user="userID",
    col_item="itemID",
    col_rating="rating",
    col_timestamp="timestamp",
    # jaccard similarity 兩個item 相似度的指標（重疊程度）
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    timedecay_formula=True,
    normalize=True
)

### 2.2 Train the SAR model on our training data, and get the top-k recommendations for our testing data

SAR first computes an item-to-item ***co-occurence matrix***. Co-occurence represents the number of times two items appear together for any given user. Once we have the co-occurence matrix, we compute an ***item similarity matrix*** by rescaling the cooccurences by a given metric (Jaccard similarity in this example). 

We also compute an ***affinity matrix*** to capture the strength of the relationship between each user and each item. Affinity is driven by different types (like *rating* or *viewing* a movie), and by the time of the event. 

Recommendations are achieved by multiplying the affinity matrix $A$ and the similarity matrix $S$. The result is a ***recommendation score matrix*** $R$. We compute the ***top-k*** results for each user in the `recommend_k_items` function seen below.

A full walkthrough of the SAR algorithm can be found [here](../02_model_collaborative_filtering/sar_deep_dive.ipynb).

In [25]:
with Timer() as train_time:
    model.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

2025-12-24 16:02:51,257 INFO     Collecting user affinity matrix
2025-12-24 16:02:51,258 INFO     Calculating time-decayed affinities
2025-12-24 16:02:51,265 INFO     Creating index columns
2025-12-24 16:02:51,292 INFO     Calculating normalization factors
2025-12-24 16:02:51,300 INFO     Building user affinity sparse matrix
2025-12-24 16:02:51,302 INFO     Calculating item co-occurrence
2025-12-24 16:02:51,377 INFO     Calculating item similarity
2025-12-24 16:02:51,377 INFO     Using jaccard based similarity
2025-12-24 16:02:51,408 INFO     Done training


Took 0.15509883400000035 seconds for training.


In [26]:
with Timer() as test_time:
    # recommend k items for each user in the test set, remove_seen=True means do not recommend items the user has already seen
    top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

print("Took {} seconds for prediction.".format(test_time.interval))

2025-12-24 16:02:51,421 INFO     Calculating recommendation scores
2025-12-24 16:02:51,473 INFO     Removing seen items


Took 0.06677804199989623 seconds for prediction.


In [27]:
# top_k means top k recommended items for each user, the higher the prediction the more likely the user will like the item
print(top_k)
top_k[top_k['userID'] == 943]

       userID  itemID  prediction
0           1     433    2.910697
1           1     204    2.906224
2           1     403    2.906136
3           1     174    2.870639
4           1      70    2.863253
...       ...     ...         ...
18855     943      88    2.956596
18856     943     173    2.950848
18857     943      95    2.931581
18858     943      77    2.931088
18859     943      71    2.928913

[18860 rows x 3 columns]


,userID,itemID,prediction
18840,943,82,3.366127
18841,943,568,3.331289
18842,943,550,3.306152
18843,943,176,3.241390
18844,943,96,3.176559
18845,943,144,3.110121
18846,943,265,3.109271
18847,943,231,3.107126
18848,943,423,3.080639
18849,943,97,3.079949


### 2.3. Evaluate how well SAR performs

We evaluate how well SAR performs for a few common ranking metrics provided in the `python_evaluation` module. We will consider the Mean Average Precision (MAP), Normalized Discounted Cumalative Gain (NDCG), Precision, and Recall for the top-k items per user we computed with SAR. User, item and rating column names are specified in each evaluation method.

In [28]:
# Ranking metrics
#this is about putting the user's favorite items on the top of the recommendation list
eval_map = map(test, top_k, col_user="userID", col_item="itemID", col_rating="rating", k=TOP_K)
eval_ndcg = ndcg_at_k(test, top_k, col_user="userID", col_item="itemID", col_rating="rating", k=TOP_K)
eval_precision = precision_at_k(test, top_k, col_user="userID", col_item="itemID", col_rating="rating", k=TOP_K)
eval_recall = recall_at_k(test, top_k, col_user="userID", col_item="itemID", col_rating="rating", k=TOP_K)


In [29]:
# Rating metrics
#compare actual ratings with predicted ratings
eval_rmse = rmse(test, top_k, col_user="userID", col_item="itemID", col_rating="rating")
eval_mae = mae(test, top_k, col_user="userID", col_item="itemID", col_rating="rating")
eval_rsquared = rsquared(test, top_k, col_user="userID", col_item="itemID", col_rating="rating")
eval_exp_var = exp_var(test, top_k, col_user="userID", col_item="itemID", col_rating="rating")


In [30]:
positivity_threshold = 2
test_bin = test.copy()
#binarize : covert the rating into 0 or 1(like or dislike) anything above 2 is 1, else 0
test_bin["rating"] = binarize(test_bin["rating"], positivity_threshold)

top_k_prob = top_k.copy()
#normalize predictions to [0, 1]
top_k_prob["prediction"] = minmax_scale(top_k_prob["prediction"].astype(float))

#caculate logloss
eval_logloss = logloss(
    test_bin, top_k_prob, col_user="userID", col_item="itemID", col_rating="rating"
)


In [31]:
print("Model:\t",
      "Top K:\t%d" % TOP_K,
      "MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall,
      "RMSE:\t%f" % eval_rmse,
      "MAE:\t%f" % eval_mae,
      "R2:\t%f" % eval_rsquared,
      "Exp var:\t%f" % eval_exp_var,
      "Logloss:\t%f" % eval_logloss,
      sep='\n')

Model:	
Top K:	20
MAP:	0.136826
NDCG:	0.367130
Precision@K:	0.263945
Recall@K:	0.266500
RMSE:	1.262854
MAE:	1.063496
R2:	-0.590732
Exp var:	0.083256
Logloss:	0.599933


In [ ]:
#basically we find all the rating in test for user 54, then sort by rating and take top k
#then compare with the recommendation(created by sar model) 
#use column "prediction" to see how well the model did
user_id = 54

# Ground truth items the user has rated in the test set(user actually liked)
#fine user which id is 54, sorted by rating, then take top k(slice)
ground_truth = test[test["userID"] == user_id].sort_values(
    by="rating", ascending=False
)[:TOP_K]
#create recommendation for the user
prediction = model.recommend_k_items(
    pd.DataFrame(dict(userID=[user_id])), remove_seen=True
)
#merge ground truth and prediction 
df = pd.merge(ground_truth, prediction, on=["userID", "itemID"], how="left")
df.head(10)

2025-12-24 16:02:51,836 INFO     Calculating recommendation scores
2025-12-24 16:02:51,837 INFO     Removing seen items


,userID,itemID,rating,timestamp,prediction
0,54,327,5.0,880928893,NaN
1,54,272,5.0,890608175,NaN
2,54,742,5.0,880934806,1.950736
3,54,147,5.0,880935959,NaN
4,54,240,4.0,880936500,NaN
5,54,258,4.0,880928745,2.100102
6,54,118,4.0,880937813,1.864598
7,54,302,4.0,880928519,NaN
8,54,307,4.0,891813846,NaN
9,54,595,3.0,880937813,NaN


Above, we see that one of the highest rated items from the test set was recovered by the model's top-k recommendations, however the others were not. Offline evaluations are difficult as they can only use what was seen previously in the test set and may not represent the user's actual preferences across the entire set of items. Adjustments to how the data is split, algorithm is used and hyper-parameters can improve the results here. 

In [33]:
# Record results for tests - ignore this cell
store_metadata("map", eval_map)
store_metadata("ndcg", eval_ndcg)
store_metadata("precision", eval_precision)
store_metadata("recall", eval_recall)
store_metadata("train_time", train_time.interval)
store_metadata("test_time", test_time.interval)